In [0]:
#This Code predict Label for SVF Issues with Title

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

data = pd.read_csv('IssueDetailsSVF_20102019.csv')
data = data.sample(frac=1)

y = list(data['Label']) 
x = list(data['Title']) 

le = preprocessing.LabelEncoder()
le.fit(y)

categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le,categories)

untest = decode(le, test)


x_enc = x
y_enc = encode(le, y)

#80% / 20% train / test split:

train_size = int(len(x) * .8)

x_train = np.asarray(x_enc[:train_size])
y_train = np.asarray(y_enc[:train_size])

x_test = np.asarray(x_enc[train_size:])
y_test = np.asarray(y_enc[train_size:])

In [0]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

In [52]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(len(categories), activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [53]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=60, batch_size=32)
    model.save_weights('./SVF_T_Label_Predict.h5')

Epoch 1/60
73/73 [==============================] - 15s 210ms/step - loss: 1.7668 - acc: 0.3288
Epoch 2/60
73/73 [==============================] - 1s 11ms/step - loss: 1.6330 - acc: 0.5068
Epoch 3/60
73/73 [==============================] - 1s 11ms/step - loss: 1.5283 - acc: 0.5068
Epoch 4/60
73/73 [==============================] - 1s 11ms/step - loss: 1.4331 - acc: 0.4932
Epoch 5/60
73/73 [==============================] - 1s 11ms/step - loss: 1.3516 - acc: 0.4932
Epoch 6/60
73/73 [==============================] - 1s 11ms/step - loss: 1.2884 - acc: 0.4932
Epoch 7/60
73/73 [==============================] - 1s 11ms/step - loss: 1.2441 - acc: 0.4932
Epoch 8/60
73/73 [==============================] - 1s 11ms/step - loss: 1.2209 - acc: 0.4932
Epoch 9/60
73/73 [==============================] - 1s 11ms/step - loss: 1.2020 - acc: 0.4932
Epoch 10/60
73/73 [==============================] - 1s 11ms/step - loss: 1.1794 - acc: 0.4932
Epoch 11/60
73/73 [==============================] - 1s 1

In [0]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./SVF_T_Label_Predict.h5')  
    predicts = model.predict(x_test, batch_size=32)  

In [55]:
y_test = np.asarray(y_enc[train_size:])
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))



              precision    recall  f1-score   support

         bug       0.50      1.00      0.67         1
 enhancement       0.00      0.00      0.00         3
 help wanted       0.00      0.00      0.00         1
     invalid       0.00      0.00      0.00         3
    question       0.58      0.64      0.61        11

    accuracy                           0.42        19
   macro avg       0.22      0.33      0.26        19
weighted avg       0.36      0.42      0.39        19



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# New Section